# **METAMODELING WITH ARTIFICIAL NEURAL NETWORK**

In this notebook, we will use the results of Abaqus analyses in order to build an Artificial Neural Network (ANN) of the Finite Element (FE) analysis solver.

In [1]:
# Install latest Tensorflow build
#!pip install -q tf-nightly-2.0-preview
from tensorflow import summary
%load_ext tensorboard

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import time
import sys
import os

In [3]:
# Scikit-learn
from sklearn.model_selection import train_test_split

# PyTorch
import torch.utils.data as Data
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.utils.tensorboard import SummaryWriter

# Pyrenn
! pip install pyrenn
import pyrenn as prn

# Matplotlib spec
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Palatino']}) # Palatino font
plt.rcParams['pdf.fonttype'] = 42

In [4]:
torch.__version__

NameError: name 'torch' is not defined

When this notebook has been generated the result of the previous line of code is: _'1.5.1+cu101'_

We fix the seed in order to obtain reproducible results.

__N.B.__ : Reproducible results are obtained every time the runtime is restarded and runned. If you run multiple time the same cell the results will not be reporducible.

In [ ]:
seed = 0
np.random.seed(seed=seed)
torch.manual_seed(seed=seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed(seed=seed)

## **Data preprocessing**

We start by importing some information about the model used to generate the dataset.

In [1]:
# Modify parameter to choose the output folder to consider
load_case = 'axial'
stacking_sequence = 'symmetric_balanced'
data_set = '2x'
fiber_path = 'harmlin'

# Check if notebook running in Colab
is_colab = 'google.colab' in sys.modules

# Model info folder
if is_colab:
    input_folder = './'
else:
    input_folder = load_case + '/' + stacking_sequence + '/' + data_set + '/' + fiber_path + '/'

info = pd.read_csv(input_folder + 'model_info.csv', sep=",")
info.index = ['Value']
eff_plies = int(info['EffectivePlies'].values)
train_smp = int(info['Train'].values)
info.head()

NameError: name 'sys' is not defined

At this point we have to import the data set containing the input and output of the FE analysis. The data is stored in a dataframe in which the upper part is associated to the training set and the lower part to the test set. The precise number of upper row belonging to the train set is indicated in the info above.

In [7]:
# Model info folder
if is_colab:
    data_folder = './'
else:
    data_folder = '../dataset/' + load_case + '/' + stacking_sequence + '/'\
                    + data_set + '/' + fiber_path + '/'

data_orig = pd.read_csv(data_folder + '/data.csv', sep=',')
data = data_orig.drop(columns='Stiffness')
data_orig

,Amplitude1,PhaseShift1,Omega1,Beta1,Amplitude2,PhaseShift2,Omega2,Beta2,Buckling,Stiffness
0,1.346,43.532,0.577,42.909,-8.553,-40.104,1.525,-6.376,292.386,298.618
1,-22.383,-71.063,1.114,27.844,29.049,-70.370,0.821,1.689,215.937,383.198
2,-35.555,52.964,0.195,-80.748,1.128,-32.291,0.035,26.998,304.865,192.210
3,-23.268,54.123,1.211,35.123,-48.020,42.979,1.229,53.272,243.675,105.103
4,-101.971,-77.838,1.118,78.264,7.348,21.548,1.688,11.832,232.324,273.162
...,...,...,...,...,...,...,...,...,...,...
145,29.920,41.062,1.695,-54.802,-125.334,-84.776,0.259,-2.603,272.834,254.537
146,35.540,-32.174,1.291,49.134,113.044,77.722,0.468,-30.086,213.910,78.865
147,-96.888,2.429,0.270,-15.823,-33.970,16.777,1.985,61.143,378.629,202.524
148,55.056,53.195,0.386,-42.375,44.873,-58.405,0.411,-24.463,247.500,231.079


After importing the data we change the sign of the features associated to negeative values of the buckling load, and the sign of the buckling loads. In this way all the critical values have the same sign.

In [8]:
# indexes = data['Buckling'].values < 0
# data[indexes] = -data[indexes]
# data

The most important step to perform before training our model is the normalization of the variables. Different strategies are possible at this end, among which 2 are the most used:

* Range normalization: converts all the values to the range $[0, 1]$

* Standard score normalization: forces the variables to have $0$ mean and $1$ standard deviation

We will try both to see the effect on the model performance.

In [9]:
def range_norm(x):
    """normalization in range [0, 1]"""
    x_min = np.min(x, axis=0)
    x_max = np.max(x, axis=0)
    x_norm = (x - x_min) / (x_max - x_min)

    return x_norm

def std_norm(x):
    """normalization with zero mean and unitary standard deviation"""
    m = np.mean(x, axis=0)
    s = np.std(x, axis=0)
    x_norm = (x - m) / s
    
    return x_norm

In [10]:
data_norm = std_norm(data_orig)
data_norm.describe()

,Amplitude1,PhaseShift1,Omega1,Beta1,Amplitude2,PhaseShift2,Omega2,Beta2,Buckling,Stiffness
count,1.500000e+02,150.000000,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,2.257453e-17,0.000000,4.921989e-16,6.013708e-17,1.221245e-17,-4.440892e-17,-8.585725e-17,1.776357e-16,6.467049e-17,-1.998401e-16
std,1.003350e+00,1.003350,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00
min,-2.237077e+00,-1.715443,-1.709088e+00,-1.710976e+00,-2.571150e+00,-1.704126e+00,-1.725275e+00,-1.708281e+00,-2.208064e+00,-1.248515e+00
25%,-4.895682e-01,-0.861636,-8.611402e-01,-8.335842e-01,-5.083211e-01,-8.601580e-01,-8.650708e-01,-8.659319e-01,-7.041367e-01,-9.208806e-01
50%,4.664685e-02,-0.001340,-1.041492e-03,-7.252163e-04,-2.542373e-02,1.044490e-02,-4.866849e-03,3.595344e-03,-2.162520e-02,-1.514433e-02
75%,4.913795e-01,0.876075,8.738117e-01,8.441662e-01,5.452075e-01,8.502479e-01,8.726363e-01,8.474090e-01,6.105571e-01,7.140474e-01
max,2.361024e+00,1.723493,1.719156e+00,1.729762e+00,2.424907e+00,1.725804e+00,1.715541e+00,1.727091e+00,3.334353e+00,2.843263e+00


Now we can split the data into training and test set. The two sets have been generate independently during the DOE.

In [11]:
X = data_norm.drop(['Buckling', 'Stiffness'], axis=1).values
Y = data_norm[['Buckling','Stiffness']].values

# Train set
_X_train = X[:train_smp, :]
_Y_train = Y[:train_smp]

# Test set
X_test = X[train_smp:, :]
Y_test = Y[train_smp:]

print('- - - - -')
print('Problem info:')
print('- - - - -')
print("X_train : {}".format(_X_train.shape))
print("Y_train : {}".format(_Y_train.shape))
print("X_test : {}".format(X_test.shape))
print("Y_test : {}".format(Y_test.shape))
print('- - - - -')

- - - - -
Problem info:
- - - - -
X_train : (120, 8)
Y_train : (120, 2)
X_test : (30, 8)
Y_test : (30, 2)
- - - - -


We can now split the training set into train and val. In this way the validation set will be used to monitoring the overfitting/underfitting.

In [12]:
X_train, X_val, Y_train, Y_val = train_test_split(_X_train, _Y_train, test_size=0.5, random_state=seed)

At this point we can generate the iterable data sets for Torch

In [13]:
# just for the training set
batch = 16

def _init_fn(worker_id):
    np.random.seed(int(seed))
    
train_dataset = Data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train).float())
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=batch, shuffle=True, num_workers=0, pin_memory=True, worker_init_fn=_init_fn)

val_dataset = Data.TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(Y_val).float())
val_loader = Data.DataLoader(dataset=val_dataset, batch_size=X_val.shape[0], shuffle=True, num_workers=0, pin_memory=True, worker_init_fn=_init_fn)

test_dataset = Data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(Y_test).float())
test_loader = Data.DataLoader(dataset=test_dataset, batch_size=X_test.shape[0], shuffle=True, num_workers=0, pin_memory=True, worker_init_fn=_init_fn)

## **Neural network**

First define network class

In [18]:
class MLPNN(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(MLPNN, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.zeros_(self.linear1.bias)
        self.linear2 = torch.nn.Linear(H, H)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        torch.nn.init.zeros_(self.linear2.bias)
        self.linear3 = torch.nn.Linear(H//2, H)
        torch.nn.init.xavier_uniform_(self.linear3.weight)
        torch.nn.init.zeros_(self.linear3.bias)
        self.linear4 = torch.nn.Linear(H, D_out)
        torch.nn.init.xavier_uniform_(self.linear4.weight)
        torch.nn.init.zeros_(self.linear4.bias)

    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        h_relu = self.linear2(h_relu).clamp(min=0)
        #h_relu = self.linear3(h_relu).clamp(min=0)
        y_pred = self.linear4(h_relu)
        return y_pred

In [15]:
class EarlyStopping:
    def __init__(self, patience=30, delta=1e-3, path='checkpoint.pt'):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        if self.val_loss_min > val_loss:
            self.val_loss_min = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.path)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [19]:
n_x = X_train.shape[1]
n_y = Y_train.shape[1]
D_in, H, D_out = n_x, 32, n_y

# %rm -rf runs
tb = SummaryWriter() # output to ./runs/ directory

modelMLP = MLPNN(D_in, H, D_out)

epochs = 1000
lr = 1e-3

is_optimizing = True

if(os.path.isfile('net_weights/weights_NN') and is_optimizing==False):
    modelMLP.load_state_dict(torch.load('net_weights/weights_NN'))
    print(modelMLP.eval())
else:
    criterion = torch.nn.MSELoss(reduction='mean') 
    optimizer = torch.optim.Adam(modelMLP.parameters(), lr=lr, weight_decay=0.01)
    train_loss = []
    for epoch in range(epochs):
        for step, (batch_x, batch_y) in enumerate(train_loader):
            y_pred = modelMLP(batch_x)
            loss = criterion(y_pred, batch_y)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            for step, (val_x, val_y) in enumerate(val_loader):
                val_pred = modelMLP(val_x)
                loss_val = criterion(val_pred, val_y)

        if epoch % 50 == 0:
            print("Iteration: ", epoch, " Loss: ", loss.item(), " Val loss: ", loss_val.item())

    torch.save(modelMLP.state_dict(), 'weights_NN')

tb.add_graph(modelMLP, batch_x)
tb.close()

Iteration:  0  Loss:  1.1381099224090576  Val loss:  1.0261197090148926
Iteration:  50  Loss:  0.3996419906616211  Val loss:  1.0528138875961304
Iteration:  100  Loss:  0.2617047131061554  Val loss:  1.0494502782821655
Iteration:  150  Loss:  0.150391086935997  Val loss:  1.1083488464355469
Iteration:  200  Loss:  0.06864530593156815  Val loss:  1.153719186782837
Iteration:  250  Loss:  0.045219164341688156  Val loss:  1.1826711893081665
Iteration:  300  Loss:  0.016674937680363655  Val loss:  1.2101542949676514
Iteration:  350  Loss:  0.020762121304869652  Val loss:  1.222291350364685
Iteration:  400  Loss:  0.01847849041223526  Val loss:  1.2347774505615234
Iteration:  450  Loss:  0.01702648587524891  Val loss:  1.2460888624191284
Iteration:  500  Loss:  0.01572049967944622  Val loss:  1.2471487522125244
Iteration:  550  Loss:  0.018908506259322166  Val loss:  1.25712251663208
Iteration:  600  Loss:  0.014085493981838226  Val loss:  1.2596080303192139
Iteration:  650  Loss:  0.010353

In [17]:
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 258), started 4:47:33 ago. (Use '!kill 258' to kill it.)

<IPython.core.display.Javascript object>